In [ ]:
import tensorflow as tf
import numpy as np
import pandas as pd

In [ ]:
data=pd.read_csv('train.csv')

In [ ]:
x=(data.iloc[:,1:785].T)
y_train=data.iloc[:,0]

In [ ]:
range1=(np.array(np.max(x,axis=1)-np.min(x,axis=1)).reshape(784,1))+1
avg1=(1/42000)*(np.sum(x,axis=1))
avg1=np.array(avg1).reshape(784,1)
x_train=(x-np.dot(avg1,np.ones((1,42000))))/np.dot(range1,np.ones((1,42000)))

In [ ]:
tf.reset_default_graph()
y_train=tf.one_hot(y_train,10)

In [ ]:
sess=tf.Session()
y_train=sess.run(y_train)
sess.close()

In [ ]:
y_train=y_train.T
x=tf.placeholder(tf.float32,shape=[x_train.shape[0],None])
y=tf.placeholder(tf.float32,shape=[y_train.shape[0],None])

In [ ]:
y_train.shape

In [ ]:
thetaBA=tf.get_variable("thetaBA",[150,784],initializer=tf.contrib.layers.xavier_initializer())
bBA=tf.get_variable("bBA",[150,1],initializer=tf.zeros_initializer())
thetaCB=tf.get_variable("thetaCB",[100,150],initializer=tf.contrib.layers.xavier_initializer())
bCB=tf.get_variable("bCB",[100,1],initializer=tf.zeros_initializer())
thetaDC=tf.get_variable("thetaDC",[10,100],initializer=tf.contrib.layers.xavier_initializer())
bDC=tf.get_variable("bDC",[10,1],initializer=tf.zeros_initializer())

In [ ]:
blayer=tf.add(tf.matmul(thetaBA,x) ,bBA)
blayer=tf.nn.relu(blayer)
clayer=tf.add(tf.matmul(thetaCB,blayer) , bCB)
clayer=tf.nn.relu(clayer)
dlayer=tf.add(tf.matmul(thetaDC,clayer) , bDC)


In [ ]:
logits1=tf.transpose(dlayer)
labels1=tf.transpose(y)
cost=tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits1,labels=labels1))

In [ ]:
optimizer=tf.train.AdamOptimizer(learning_rate=0.001).minimize(cost)

In [ ]:
init=tf.global_variables_initializer()
sess=tf.Session()
sess.run(init)
for i in range(1,400):
    train_data=sess.run(optimizer,feed_dict={x:x_train,y:y_train})
    cost1= sess.run(cost,feed_dict={x:x_train,y:y_train})
    print(cost1,end=" ")
    print(i,end=" ")
sess.close()
    

In [ ]:
vid=cv2.VideoCapture(0)
vid.set(3, 280)
vid.set(4, 280)

while True:
    
    check,frame=vid.read()
    gray=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
    a1=rescale_frame(gray,percent=10)
    a=cv2.resize(a1,(28,28))
    a=np.reshape(a,(784,1))
    a=(a-avg1)/(range1)
    abc=sess.run(dlayer,feed_dict={x:a})
    abc=np.array(abc.argmax(axis=0))

    print(abc,end="")
    cv2.imshow("a",frame)
    key=cv2.waitKey(10)
    if (key==ord('q')):
        break;

vid.release()
cv2.destroyAllWindows()


In [ ]:
def rescale_frame(frame, percent=75):
    width = int(frame.shape[1] * percent/ 100)
    height = int(frame.shape[0] * percent/ 100)
    dim = (width, height)
    return cv2.resize(frame, dim, interpolation =cv2.INTER_AREA)q